# Project 1 - Final
Ankit Dey

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn import set_config
set_config(display='diagram')

In [2]:
sales_predictions = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSwXLcK1gHn4S6xdVnTN1kZIAdiR3EGWqJKzxOoqEKOdk--tVrbdN_aIQN-PgCVUSyMU_R7KSHsGkOV/pub?gid=135014557&single=true&output=csv'
df = pd.read_csv(sales_predictions)

In [3]:
# Explore dataset
display(df.info())
display(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


None

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [4]:
# Check for duplicates
df.duplicated().sum()

0

In [5]:
# Identify missing values
display(df.isna().sum())
print('\n')
print(f"Item_Weight column is missing {round((df['Item_Weight'].isna().sum() / len(df['Item_Weight']) * 100), 1)}% of its rows.")
print(f"Outlet_Size column is missing {round((df['Outlet_Size'].isna().sum() / len(df['Outlet_Size']) * 100), 1)}% of its rows.")
print(f"Item_Weight column contains {len(df['Item_Weight'].dropna())} non-missing values.")
print(f"Outlet_Size column contains {len(df['Outlet_Size'].dropna())} non-missing values.")

# Item_Weight and Outlet_Size columns are missing a moderate percentage of data, but still contain enough data points for column statistics to be useful
# Item_Weight and Outlet_Size may be correlated to sales -> columns should not be dropped
# Values will be imputed after train-test split to prevent data leakage in our model

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64



Item_Weight column is missing 17.2% of its rows.
Outlet_Size column is missing 28.3% of its rows.
Item_Weight column contains 7060 non-missing values.
Outlet_Size column contains 6113 non-missing values.


In [6]:
# Identify syntax errors in categorical data
data_types = df.dtypes
str_cols = data_types[data_types == 'object'].index

for col in str_cols:
  print(f'- {col}:')
  print(df[col].value_counts(dropna=False))
  print('\n\n')

- Item_Identifier:
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64



- Item_Fat_Content:
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64



- Item_Type:
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64



- Outlet_Identifier:
OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    9

In [7]:
# Fix syntax errors in Item_Fat_Content column
df['Item_Fat_Content'] = df['Item_Fat_Content'].str.replace('LF', 'Low Fat')
df['Item_Fat_Content'] = df['Item_Fat_Content'].str.replace('low fat', 'Low Fat')
df['Item_Fat_Content'] = df['Item_Fat_Content'].str.replace('reg', 'Regular')
df['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular'], dtype=object)

In [8]:
# Identify errors in numerical data
df.describe() # No errors

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [9]:
# Define features matrix and target vector
X = df.drop(columns=['Item_Outlet_Sales', 'Outlet_Establishment_Year'])
y = df['Item_Outlet_Sales']

# Split data into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [10]:
# Instantiate transformers
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore')

mean_imputer = SimpleImputer(strategy='mean')
constant_imputer = SimpleImputer(strategy='constant', fill_value='missing')
most_frequent_imputer = SimpleImputer(strategy='most_frequent')

In [11]:
# Instantiate ordinal encoder
item_fat_content_labels = ['Low Fat', 'Regular']
outlet_size_labels = ['Small', 'Medium', 'High']

ordered_labels = [item_fat_content_labels, outlet_size_labels]

ordinal_encoder = OrdinalEncoder(categories=ordered_labels)

In [12]:
# Instantiate pipelines for nominal, ordinal, and numerical data
number_pipe = make_pipeline(mean_imputer, scaler) # Mean imputer is used, rather than median imputer, for numerical data because there are no great outliers
                                                  # that would skew the mean
nominal_pipe = make_pipeline(constant_imputer, ohe) # Constant 'missing' imputer is used for nominal data to identify correlations with missing values
ordinal_pipe = make_pipeline(most_frequent_imputer, ordinal_encoder) # Most frequent imputer is used for ordinal data since the imputed value can be ordered

In [13]:
# Create tuples to match transformations with columns
num_selector = make_column_selector(dtype_include='number')
ord_cols = ['Item_Fat_Content', 'Outlet_Size']
nom_cols = ['Item_Identifier', 'Item_Type', 'Outlet_Identifier', 'Outlet_Location_Type', 'Outlet_Type']

num_tuple = (number_pipe, num_selector)
ord_tuple = (ordinal_pipe, ord_cols)
nom_tuple = (nominal_pipe, nom_cols)

In [14]:
# Instantiate column transformer
preprocessor = make_column_transformer(ord_tuple,
                                       nom_tuple,
                                       num_tuple,
                                       remainder='drop')
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Low '
                                                                              'Fat',
                                                                              'Regular'],
                                                                             ['Small',
                                                                              'Medium',
                                                                              'High']]))]),
                                 ['Item_Fat_Content', 'Outlet_Size']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Item_Identifier', 'Item_Type',
                                  'Outlet_Identifier', 'Outlet_Location_Type',
                                  'Outlet_Type']),
                                ('pipeline-3',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fdf81296850>)])

In [15]:
# Fit preprocessor then transform train and test data
preprocessor.fit(X_train)

X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [16]:
# Instantiate and fit linear reg model
lin_reg = LinearRegression()

lin_reg.fit(X_train_processed, y_train)

LinearRegression()

In [17]:
# Define evaluate function to return R^2, MAE, and RMSE metrics of predictions
def evaluate(y_true, y_preds):
    r2 = r2_score(y_true, y_preds)
    mae = mean_absolute_error(y_true, y_preds)
    rmse = np.sqrt(mean_squared_error(y_true, y_preds))

    print(f'R^2: {r2}')
    print(f'MAE: {mae}')
    print(f'RMSE: {rmse}')

In [18]:
# Evaluate linear reg model
lin_train_preds = lin_reg.predict(X_train_processed)
lin_test_preds = lin_reg.predict(X_test_processed)

print('Train')
evaluate(y_train, lin_train_preds)
print('\nTest')
evaluate(y_test, lin_test_preds)

Train
R^2: 0.6716977904752328
MAE: 735.6588224619135
RMSE: 985.6956156881015

Test
R^2: 0.38335521013416973
MAE: 967.2683704491039
RMSE: 1304.3423514875735


In [19]:
# Instantiate and fit decision tree model
dec_tree = DecisionTreeRegressor()

dec_tree.fit(X_train_processed, y_train)

DecisionTreeRegressor()

In [20]:
# Evaluate decision tree model
dec_train_preds = dec_tree.predict(X_train_processed)
dec_test_preds = dec_tree.predict(X_test_processed)

print('Train')
evaluate(y_train, dec_train_preds)
print('\nTest')
evaluate(y_test, dec_test_preds)

Train
R^2: 1.0
MAE: 1.0671480386885109e-16
RMSE: 4.925864104892086e-15

Test
R^2: 0.2507211074833783
MAE: 974.9408442045988
RMSE: 1437.7911693588478


In [21]:
# Get max depth of dec tree
dec_tree.get_depth()

60

In [22]:
dec_tree.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [23]:
# Instantiate pipeline containing preprocessor and decision tree
dec_tree_processor = make_pipeline(preprocessor, dec_tree)

In [26]:
# Tune decision tree model
depths = [x for x in range(1,61)]
min_samples_leaf_range = [x for x in range(1,100,10)]
params = [{'decisiontreeregressor__max_depth': depths,
           'decisiontreeregressor__min_samples_leaf': min_samples_leaf_range}]

gs_dec = GridSearchCV(dec_tree_processor, # use pipeline containing preprocessor and model to prevent data leakage between folds
                      param_grid=params,
                      scoring='r2',
                      cv=5)
gs_dec.fit(X_train, y_train)

print(gs_dec.best_params_)

{'decisiontreeregressor__max_depth': 6, 'decisiontreeregressor__min_samples_leaf': 51}


In [27]:
# Instantiate and fit final dec tree
tuned_dec_tree = DecisionTreeRegressor(max_depth=6, min_samples_leaf=51)

tuned_dec_tree.fit(X_train_processed, y_train)

DecisionTreeRegressor(max_depth=6, min_samples_leaf=51)

In [28]:
# Evaluate tuned dec tree using R^2, MAE, and RMSE metrics
tuned_dec_train_preds = tuned_dec_tree.predict(X_train_processed)
tuned_dec_test_preds = tuned_dec_tree.predict(X_test_processed)

print('Train')
evaluate(y_train, tuned_dec_train_preds)
print('\nTest')
evaluate(y_test, tuned_dec_test_preds)

Train
R^2: 0.6091099589297058
MAE: 754.9714801107035
RMSE: 1075.556489398908

Test
R^2: 0.5956859696863335
MAE: 734.7008404865791
RMSE: 1056.169127922691


The tuned decision tree model is recommended over the linear regression model, since the decision tree model shows a higher R^2 (0.60 vs 0.38), lower MAE (735 vs 995), and lower RMSE (1056 vs 1304) on test data. The decision tree model also shows a train R^2 of 0.61 that is very close to the test R^2 of 0.60, indicating adequate model fit.